# Phase 2: Simple Neural Networks

Tên và MSSV của từng thành viên:
- Đinh Viết Lợi - 22120188.
- Nguyễn Trần Lợi - 22120190.
- Nguyễn Nhật Long - 22120194.


## Nắm yêu cầu của Phase 2:
Đây là bài toán phân loại nhận biết đoạn âm thanh có chứa tiếng gà tây hay không bằng cách sử dụng Simple Neural Network và tuân theo quy trình: Phân tích Dữ liệu Khám phá (Exploratory Data Analysis), Phát triển Mô hình (Model Development) và Đánh giá Mô hình (Model Evaluation).

## Import các thư viện cần thiết

In [73]:
import json
import numpy as np
import pandas as pd
import time
import tracemalloc
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


## Đọc dữ liệu

In [74]:
# Load dữ liệu
with open('Datasets/train.json', 'r') as file:
    train_data = json.load(file)
with open('Datasets/test.json', 'r') as file:
    test_data = json.load(file)

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [75]:
# Dữ liệu huấn luỵện
train_data.head()

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0


In [76]:
# Dữ liệu test
test_data.head()

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30


## Khám phá dữ liệu

### Các cột có trong dữ liệu

In [77]:
train_data.columns

Index(['audio_embedding', 'is_turkey', 'vid_id',
       'end_time_seconds_youtube_clip', 'start_time_seconds_youtube_clip'],
      dtype='object')

- Dữ liệu trong `train_df` chứa các cột trên. Trong đó, các cột cần thiết để thực hiện huấn luyện là `audio_embedding`, `is_turkey`.

In [78]:
test_data.columns

Index(['audio_embedding', 'vid_id', 'end_time_seconds_youtube_clip',
       'start_time_seconds_youtube_clip'],
      dtype='object')

- `test_df` có các cột trên. Dựa vào yêu cầu, các cột sử dụng là `vid_id`, `audio_embedding`.

## Phát triển mô hình

### Xử lý dữ liệu trước khi huấn luyện

- Vì mỗi `audio_embedding` có số lượng frame khác nhau do đó để dễ dàng trong việc huấn luyện ta thực hiện lấy trung bình của mỗi cột trong các `audio_embedding`.

In [79]:
def combined_embeddings(embeddings): # Hàm dùng để tính toán trên embeddings
    X= np.array(embeddings)
    return np.mean(X, axis=0)

In [80]:
# Tiền xử lý
train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0))) # Lấy trung bình của mỗi cột trong các audio_embedding
train_Y = train_data['is_turkey'].values # Lấy nhãn của dữ liệu huấn luyện

valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0))) # Lấy trung bình của mỗi cột trong các audio_embedding

In [81]:
# In ra kích thước của dữ liệu sau khi xử lý
print("Train X shape:", train_X.shape)

# In ra hai dòng đầu tiên của dữ liệu huấn luyện
print(train_X[:2])

Train X shape: (1195, 128)
[[166.6  29.3 198.4 106.4 215.2  57.5  77.3  65.  188.5 149.7  77.3  71.2
  120.  156.1  70.7 105.8 101.1 131.5 175.6  90.7  25.  213.2 106.5 135.5
   47.8 110.3 190.6 183.  104.5  35.6  99.6 109.2 160.  101.   99.5 141.8
  134.9  95.6  90.7  98.6  70.3 150.2  15.  160.5 196.4 140.2 113.9  62.7
   91.2 253.3 184.5  63.5  73.2 155.   30.2  50.1 147.1  38.1 119.5 197.7
  182.6  74.5  59.7 182.9 213.4 187.9 176.7 115.9 251.3 121.2 119.   86.3
  194.4 200.2  50.8 101.1 117.4 109.6 127.5 193.2 148.7 211.7 145.9 160.7
  210.   95.  113.3 232.8  44.2   0.  108.9 221.4 209.5 162.5  91.8 252.5
   98.2   6.1 205.  174.8 126.7  14.3 159.2 119.7  87.7 224.9 120.8 100.
   96.3  99.5 212.3  61.7 102.4 147.9  94.8 190.5 144.5 205.8   1.9  85.2
   44.3 186.2   0.   48.2 238.6 138.5 101.  255. ]
 [173.3  13.7 164.7  93.5 201.3  94.2 122.  114.  166.  181.8  89.5 145.3
   77.  160.1  65.3 105.1 128.4 113.5 121.2 131.3  24.3 201.  104.5  84.5
   93.8 120.4 171.9 179.8 132.3  96

In [82]:
# In ra kích thước của dữ liệu sau khi xử lý
print("Train X shape:", test_X.shape)

# In ra hai dòng đầu tiên của dữ liệu huấn luyện
print(test_X[:2])

Train X shape: (1196, 128)
[[169.7  22.2 220.3 130.3 216.3  85.9 116.3  80.6 127.9 188.8  83.   85.
  106.   79.  104.5  99.3  98.1 152.  201.5  75.   55.5 252.4  60.5  97.1
    4.5 112.1  39.9 168.9  84.8  75.2 157.8  95.2  75.1 115.7 212.1 184.5
   99.6  74.3  56.3 195.4  59.1 125.9  83.  229.  220.1  87.4  85.4  99.8
  106.8 233.6 199.1  16.2 167.8 120.9 111.  108.9  33.8  44.  118.4 186.2
  145.9 167.   92.4 130.  252.6 188.3 205.8 138.  138.5  24.  170.2  84.1
  223.1 177.1  95.5  39.6 168.9 230.7  32.1 229.5  31.4 180.1 232.8 191.8
  220.4  55.7 166.4 239.  133.9 106.7  80.4 210.6 182.2 223.7  89.2 255.
    0.   38.1 202.6 167.1  77.7 152.7 181.6 140.9 145.3 143.5  80.7 195.2
  172.9 134.4  66.8 122.1 106.9 155.5 157.3 182.1  92.2 248.8   5.8 216.3
   35.7 196.4 153.    8.3 249.8 135.5  60.5 255. ]
 [165.3  16.1 192.5 140.7 200.4  85.   65.1  49.4 161.5 182.1  70.6  70.4
  129.9 120.7  62.  108.9 100.2 114.7 224.8 108.1  60.4 239.4 101.  105.4
   21.9 129.1  96.3 248.5 122.6  63.

$\rightarrow$ Dữ liệu sau khi xử lý có kích thước là (1195,128) dối với dữ liệu huấn luyện và (1196,128) đối với dữ liệu test.

- Chuẩn hóa dữ liệu để giúp cho mô hình học dữ liệu tốt hơn.

In [83]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)

In [84]:
# In ra hai dòng đầu tiên của dữ liệu huấn luyện đã chuẩn hoá
print(Z[:2])

[[0.83290088 0.14174379 0.71166582 0.42799678 0.82513181 0.24687656
  0.23238792 0.3068933  0.67257509 0.56181665 0.35986965 0.27926159
  0.40450928 0.62067594 0.30123562 0.43275613 0.33895985 0.54476351
  0.68724778 0.3151313  0.10351967 0.83607843 0.40528634 0.58238884
  0.18745098 0.4375531  0.74745098 0.69217614 0.40286055 0.16355283
  0.41191067 0.49456522 0.68715596 0.44008715 0.36737185 0.52768166
  0.52922715 0.37936508 0.35568627 0.42068361 0.31752484 0.59367589
  0.0595002  0.60785953 0.80123203 0.54980392 0.44864599 0.22584541
  0.3902439  0.99322169 0.68680586 0.25522508 0.31647211 0.61597418
  0.11843137 0.19888845 0.57487159 0.14941176 0.47991968 0.74690813
  0.70497148 0.29353822 0.26783311 0.7164766  0.83686275 0.73686275
  0.69294118 0.46505926 0.98467909 0.47905138 0.46666667 0.33387689
  0.74278438 0.76027997 0.20031546 0.42177722 0.46039216 0.42958023
  0.50236593 0.75630915 0.63710368 0.82898894 0.54178916 0.61883589
  0.82318271 0.37254902 0.44278411 0.91041162 0.

In [85]:
# In ra hai dòng dữ liệu test sau khi chuẩn hóa
print(test_Z[:2])

[[0.84898806 0.10716025 0.82322975 0.52413516 0.82996485 0.3888056
  0.41079597 0.38054769 0.3741999  0.72624054 0.38640596 0.33346426
  0.34261715 0.31411531 0.44524925 0.40356305 0.32527372 0.63133446
  0.79176755 0.24968737 0.22981366 0.98980392 0.22106528 0.41499564
  0.01764706 0.44519966 0.15647059 0.63189397 0.32459277 0.34548239
  0.65260546 0.43115942 0.29770642 0.50413943 0.82546786 0.71237024
  0.39074147 0.29484127 0.22078431 0.84487292 0.26693767 0.49762846
  0.32923443 0.89423077 0.89856263 0.3427451  0.33638602 0.37520129
  0.45699615 0.91467305 0.75166593 0.06511254 0.72546476 0.47841872
  0.43529412 0.43231441 0.12722244 0.17254902 0.47550201 0.69611307
  0.55541972 0.65799842 0.41453567 0.50845458 0.99058824 0.73843137
  0.80705882 0.55537393 0.51759834 0.09486166 0.66745098 0.32490828
  0.86460102 0.6592301  0.37657729 0.16520651 0.66235294 0.9046685
  0.12618297 0.89944795 0.13453299 0.70418641 0.90676186 0.74454325
  0.86404715 0.21843137 0.65159261 0.9354318  0.52

### Chia tỉ lệ huấn luyện

In [86]:
# Chia train / validation
train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.3, random_state=97)

### Khởi tạo danh sách các mô hình tối ưu nhất

In [87]:
# Danh sách lưu các mô hình thỏa điều kiện
qualified_models = []

### Huấn luyện, model thỏa tiêu chí được lưu lại, in ra thời gian, bộ nhớ và các chỉ số. 

##### Vì kết quả mỗi lần huấn luyện model là khác nhau, ta huấn luyện model 200 lần để xem model nào cho thông số cao nhất thì lưu lại

##### **Nhận thấy:** Khi **Precision** và **Accuracy** từ 0.96 trở lên thì **Score** thường đạt được khá cao. Đặc biệt là **Precision** càng cao thì **Score** thường càng cao. Do đó ta chỉ lưu lại những model thỏa điều này.

In [88]:
# Huấn luyện
for i in range(200):
    print(f"\n▶️ Huấn luyện lần {i + 1}")

    start_time = time.time()
    tracemalloc.start()

    model = Sequential([
        Input(shape=(train_Z.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['auc'])

    early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

    model.fit(
        train_Z, train_Y,
        validation_data=(val_Z, val_Y),
        epochs=80,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )

    # Dự đoán validation
    y_pred_prob = model.predict(val_Z).ravel()
    y_pred = (y_pred_prob >= 0.5).astype(int)

    auc = roc_auc_score(val_Y, y_pred_prob)
    acc = accuracy_score(val_Y, y_pred)
    prec = precision_score(val_Y, y_pred)
    rec = recall_score(val_Y, y_pred)
    f1 = f1_score(val_Y, y_pred)

    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    if prec >= 0.96 and acc >= 0.96:
        print(f"AUC       : {auc:.4f}")
        print(f"Accuracy  : {acc:.4f}")
        print(f"Precision : {prec:.4f}")
        print(f"Recall    : {rec:.4f}")
        print(f"F1 Score  : {f1:.4f}")
        print(f"⏱ Thời gian huấn luyện: {end_time - start_time:.2f} giây")
        print(f"📦 Bộ nhớ đỉnh sử dụng: {peak / 1024 / 1024:.2f} MB")

        test_pred = model.predict(test_Z).ravel()

        qualified_models.append({
            "model": model,
            "precision": prec,
            "auc": auc,
            "acc": acc,
            "recall": rec,
            "f1": f1,
            "time": end_time - start_time,
            "memory": peak / 1024 / 1024,
            "test_pred": test_pred
        })


▶️ Huấn luyện lần 1
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

▶️ Huấn luyện lần 2
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

▶️ Huấn luyện lần 3
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
AUC       : 0.9942
Accuracy  : 0.9666
Precision : 0.9860
Recall    : 0.9338
F1 Score  : 0.9592
⏱ Thời gian huấn luyện: 4.69 giây
📦 Bộ nhớ đỉnh sử dụng: 3.60 MB
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

▶️ Huấn luyện lần 4
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

▶️ Huấn luyện lần 5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

▶️ Huấn luyện lần 6
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

▶️ Huấn luyện lần 7
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

▶️ Huấn luyện lần 8
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

▶️ Huấn luyện lần 9
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
AUC       : 0.9939
Accuracy  : 0.9610
Precision : 0.9724
Recall    : 0.9338
F1 Score  : 0.9527
⏱ Thời gian huấn luyện: 4.89 giây
📦 Bộ nhớ đỉnh sử dụng: 3.61 MB
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

▶️ Huấn luyện lần 10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

### Trong các model thỏa tiêu chí, chọn ra 3 model có precision cao nhất

In [89]:
# Sắp xếp theo precision giảm dần
qualified_models = sorted(qualified_models, key=lambda x: x["precision"], reverse=True)

# Lưu 3 mô hình có precision cao nhất
top_models = qualified_models[:3]

# In thông tin các mô hình được lưu
for idx, m in enumerate(top_models):
    print(f"\n✅ model{idx + 1} (Precision: {m['precision']:.4f}):")
    print(f"AUC      : {m['auc']:.4f}")
    print(f"Accuracy : {m['acc']:.4f}")
    print(f"Precision: {m['precision']:.4f})")
    print(f"Recall   : {m['recall']:.4f}")
    print(f"F1 Score : {m['f1']:.4f}")
    print(f"⏱ Time   : {m['time']:.2f} giây")
    print(f"📦 Memory : {m['memory']:.2f} MB")


✅ model1 (Precision: 0.9860):
AUC      : 0.9942
Accuracy : 0.9666
Precision: 0.9860)
Recall   : 0.9338
F1 Score : 0.9592
⏱ Time   : 4.69 giây
📦 Memory : 3.60 MB

✅ model2 (Precision: 0.9792):
AUC      : 0.9928
Accuracy : 0.9638
Precision: 0.9792)
Recall   : 0.9338
F1 Score : 0.9559
⏱ Time   : 3.64 giây
📦 Memory : 3.51 MB

✅ model3 (Precision: 0.9792):
AUC      : 0.9934
Accuracy : 0.9638
Precision: 0.9792)
Recall   : 0.9338
F1 Score : 0.9559
⏱ Time   : 4.09 giây
📦 Memory : 3.47 MB


### Dùng model có precision cao nhất để dự đoán với tập test

In [90]:
if top_models:
    test_data['is_turkey'] = -1.0
    test_data.loc[valid_idx, 'is_turkey'] = top_models[0]['test_pred']
    test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)
else:
    print("\n❌ Không có mô hình nào đạt yêu cầu.")